In [1]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
conf = SparkConf().setMaster("local[1]").setAppName("ex37")
sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName("ex37").getOrCreate()

25/01/10 09:50:45 WARN Utils: Your hostname, paloma-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/01/10 09:50:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/10 09:50:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
schema = "sensorId STRING, date STRING, PM10 FLOAT"
data = spark.read.csv("/home/paloma/Downloads/jupyter_spark/ex37/input.csv", header=False, schema=schema)
data.show()

+--------+----------+----+
|sensorId|      date|PM10|
+--------+----------+----+
|      s1|2016-01-01|20.5|
|      s2|2016-01-01|30.1|
|      s1|2016-01-02|60.2|
|      s2|2016-01-02|20.4|
|      s1|2016-01-03|55.5|
|      s2|2016-01-03|52.5|
+--------+----------+----+



In [4]:
data = data.rdd # transforma em rdd
data.collect()

[Row(sensorId='s1', date='2016-01-01', PM10=20.5),
 Row(sensorId='s2', date='2016-01-01', PM10=30.100000381469727),
 Row(sensorId='s1', date='2016-01-02', PM10=60.20000076293945),
 Row(sensorId='s2', date='2016-01-02', PM10=20.399999618530273),
 Row(sensorId='s1', date='2016-01-03', PM10=55.5),
 Row(sensorId='s2', date='2016-01-03', PM10=52.5)]

In [18]:
# dividindo o rdd pra cada sensor
s1 = data.filter(lambda line: line.sensorId == 's1')
s2 = data.filter(lambda line: line.sensorId == 's2')

In [19]:
s1.collect()

[Row(sensorId='s1', date='2016-01-01', PM10=20.5),
 Row(sensorId='s1', date='2016-01-02', PM10=60.20000076293945),
 Row(sensorId='s1', date='2016-01-03', PM10=55.5)]

In [20]:
s2.collect()

[Row(sensorId='s2', date='2016-01-01', PM10=30.100000381469727),
 Row(sensorId='s2', date='2016-01-02', PM10=20.399999618530273),
 Row(sensorId='s2', date='2016-01-03', PM10=52.5)]

In [21]:
# arredondando valores
s1 = s1.map(lambda line: (line.sensorId, round(line.PM10, 1)))
s2 = s2.map(lambda line: (line.sensorId, round(line.PM10, 1)))

In [22]:
s1.collect()

[('s1', 20.5), ('s1', 60.2), ('s1', 55.5)]

In [23]:
s2.collect()

[('s2', 30.1), ('s2', 20.4), ('s2', 52.5)]

In [24]:
s1_max = s1.map(lambda line: line[1]).max()
s2_max = s2.map(lambda line: line[1]).max()

In [25]:
print(s1_max)

60.2


In [26]:
print(s2_max)

52.5


In [36]:
# obtendo valores filtrados do s1 e unindo com s2
sensors_max = s1.filter(lambda line: line[1] == s1_max).union(s2.filter(lambda line: line[1] == s2_max))

In [37]:
sensors_max.collect()

[('s1', 60.2), ('s2', 52.5)]